# Suspicious transanctions

This work analyzes suspicious bank transactions related to bitcoin transactions. The datasets were taken from the suggested links

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib
from matplotlib import pyplot as plt
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestNeighbors
import warnings
import time

warnings.filterwarnings('ignore')

Download dataset. Some rows were broken, so it was needed to use an error_bad_lines key

In [2]:
data_btc_tx = pd.read_csv('btc-tx.csv', sep = ',',error_bad_lines=False)

b'Skipping line 12136: expected 5 fields, saw 13\n'


Let's transform a time column to a date data type for a follow processing

In [3]:
data_btc_tx.time = pd.to_datetime(data_btc_tx.time)

In [4]:
data_btc_tx.drop('Hash', axis = 1, inplace = True)

After considering of data I decided to drop the Hash column as this column was uninformative and took too much place

In [5]:
data_btc_tx.head(2)

,time,Sender,Receiver,Transaction_amount_BTC
0,2015-03-31,1BtNH4rtj3PVe6PWYsziQMnXofZmoPmGHu,1PrGFmmFFeD6v9ihS74ewjAzSMmQxsQAuw,5272.9999
1,2015-03-31,1NE3QFkf4aFUgCNTvGjiZsY59yDsGratiU 1Bst4GwKrN2...,18zXyBVHCU3hu3ivszZNh3NqA1eA1jVBsm,4509.87714053


In [6]:
data_map = pd.read_csv('download_transactions_map.csv')
data_map.dropna(inplace = True)

Make the same actions as for the prioir dataset

In [7]:
data_map.end_date = pd.to_datetime(data_map.end_date)
data_map.begin_date = pd.to_datetime(data_map.begin_date)

In [8]:
data_map.head(2)

,id,icij_sar_id,filer_org_name_id,filer_org_name,begin_date,end_date,originator_bank_id,originator_bank,originator_bank_country,originator_iso,beneficiary_bank_id,beneficiary_bank,beneficiary_bank_country,beneficiary_iso,number_transactions,amount_transactions
0,223254,3297,the-bank-of-new-york-mellon-corp,The Bank of New York Mellon Corp.,2015-03-25,2015-09-25,cimb-bank-berhad,CIMB Bank Berhad,Singapore,SGP,barclays-bank-plc-london-england-gbr,Barclays Bank Plc,United Kingdom,GBR,68.0,5.689852e+07
1,223255,3297,the-bank-of-new-york-mellon-corp,The Bank of New York Mellon Corp.,2015-03-30,2015-09-25,cimb-bank-berhad,CIMB Bank Berhad,Singapore,SGP,barclays-bank-plc-london-england-gbr,Barclays Bank Plc,United Kingdom,GBR,118.0,1.162384e+08


Also, let's deminish count of columns in this dataset as they in general duplicate another columns

In [9]:
data_map.drop(['originator_iso','beneficiary_iso','number_transactions','icij_sar_id','filer_org_name_id','originator_bank_id','beneficiary_bank_id'], axis = 1, inplace = True)

In [11]:
data_conn = pd.read_csv('download_bank_connections.csv')
data_conn.drop(['filer_org_name_id', 'entity_b_id'], axis = 1, inplace = True)

In [12]:
data_conn.head(2)

,icij_sar_id,filer_org_name,entity_b,entity_b_country,entity_b_iso_code
0,4132,Standard Chartered Plc,Habib Metropolitan Bank Limited,Pakistan,PAK
1,3181,Standard Chartered Plc,Standard Chartered Bank,United Arab Emirates,ARE


As the first known Bitcoin transaction is dated from 2015-01-02 let's consider bank transactions from this date

In [13]:
dates = ['2016','2017']
btc_price = pd.read_csv('btc_usd_2015.csv')
for date in dates:
    btc_year = pd.read_csv(f'btc_usd_{date}.csv')
    btc_price = pd.concat([btc_price, btc_year])

In [15]:
btc_price.head(2)

,time,price
0,01.01.2015,316.910476
1,02.01.2015,315.218079


Here the same action, transforming a time column  to a  date type

In [16]:
btc_price.time = pd.to_datetime(btc_price.time)

The first dataset is consist of count of bitcoins in transaction. So I decided to merge two datasets: dataset with Bitcoin transactions and  bitcon prices and get a price in USD. This action will allow to find suspicious bank transactions related with bitcoin transactions

In [17]:
btc_data = pd.merge(data_btc_tx, btc_price,how = 'left')

A column Sender and a column Receiver in the first dataset are represented by encoded sequences. It is inconvinient to work with these long sequences, so let's decode all aliases using Label encoder

In [18]:
encoder_sender = LabelEncoder()

In [19]:
sender_encode = encoder_sender.fit_transform(btc_data.Sender.values)

In [20]:
btc_data.Sender = sender_encode

In [21]:
enc_receiver = LabelEncoder()
receiver_encode = enc_receiver.fit_transform(btc_data.Receiver.values)

In [22]:
btc_data.Receiver = receiver_encode

At the next step let's get a full price in USD for bitcoin transactions. But there is a issue, a column "Transaction_amount_BTC" consists of object type data. So let's  convert them to a float data type to get a summary result. Unfortunately some meanings can't be present as a float type. The next cycle allows to get indexes of these meaning.

In [23]:
new_btc = []
index_btc = []
mistakes = []
for num, number in enumerate(btc_data.Transaction_amount_BTC):
    try:
        new_btc.append(np.float64(number))
    except:
        mistakes.append(num)

In [24]:
btc_data.iloc[mistakes].Transaction_amount_BTC

8507           1 "
9043      \r\n20 "
9733           d "
10635     \r\n20 "
10647     \r\n20 "
10761     \r\n20 "
10906          d "
11647    \r\n200 "
25465     \r\n20 "
26231     \r\n20 "
28606     \r\n20 "
Name: Transaction_amount_BTC, dtype: object

As we can see, these meanings are rather small or unknown, so let's drop these rows from dataset

In [25]:
delete = btc_data.iloc[mistakes].index
btc_data.drop(delete, axis = 0, inplace = True)

In [26]:
btc_data.drop('Transaction_amount_BTC', axis = 1, inplace = True)

In [27]:
btc_data['Trans_amount_BTC'] = new_btc

In [28]:
btc_data.head(3)

,time,Sender,Receiver,price,Trans_amount_BTC
0,2015-03-31,64914,267373,247.031099,5272.999900
1,2015-03-31,132792,85982,247.031099,4509.877141
2,2015-03-31,21756,155511,247.031099,4138.094249


At the next step, let's calculate a full price for every bitcoin transaction in  the dataset. These meaning will be the first feature for our model

In [29]:
btc_data['btc_to_usd'] = btc_data['Trans_amount_BTC'] * btc_data['price']

To compare bank transactions and bitcoin transactions let's use two features: amount in USD (was calculated at the prior step for bitcoin transactions, for bank transactions this feature is provided by default) and a transaction date. For our model we can't use date data type, so let's convert all dates to UNIX time, and these meanings will user as the second feature

In [30]:
data_map['unix'] = data_map.begin_date.apply(lambda x: x.timestamp())   
btc_data['unix'] = btc_data.time.apply(lambda x: x.timestamp())

In [31]:
data_map.head(2)

,id,filer_org_name,begin_date,end_date,originator_bank,originator_bank_country,beneficiary_bank,beneficiary_bank_country,amount_transactions,unix
0,223254,The Bank of New York Mellon Corp.,2015-03-25,2015-09-25,CIMB Bank Berhad,Singapore,Barclays Bank Plc,United Kingdom,5.689852e+07,1.427242e+09
1,223255,The Bank of New York Mellon Corp.,2015-03-30,2015-09-25,CIMB Bank Berhad,Singapore,Barclays Bank Plc,United Kingdom,1.162384e+08,1.427674e+09


In [32]:
btc_data.head(2)

,time,Sender,Receiver,price,Trans_amount_BTC,btc_to_usd,unix
0,2015-03-31,64914,267373,247.031099,5272.999900,1.302595e+06,1.427760e+09
1,2015-03-31,132792,85982,247.031099,4509.877141,1.114080e+06,1.427760e+09


For finding Bitcoin transactions among suspicious bank transactions let's use  a Nearest Neighbours algorithm. This algorithm is able to find the most similar transactions in our feature dimension. This algorithm will be learnt on bitcoin transactions data. After learning algorithm will be able to find the nearest bank transaction.

In [33]:
nbrs = NearestNeighbors(n_neighbors=1).fit(btc_data[['btc_to_usd','unix']].values)

The next cycle works quite slowly, but it helps to extract the nearest transactions. Let's restrict out search with the following boundaries: difference between transactions is 2 days and difference between amounts is 30 000 USD (as a Bitcoin course is too changeable)

In [34]:
similar = {}
for number, trans in enumerate(data_map[['amount_transactions','unix']].values):
    trans = trans.reshape(-1,2)
    _, indexes = nbrs.kneighbors(trans)
    for index in indexes[0]:
        if abs(data_map.iloc[number].begin_date - btc_data.iloc[index].time).days <= 1 and abs(data_map.iloc[number].amount_transactions - btc_data.iloc[index].btc_to_usd) < 30000:
            similar[number] = index

So, we got indexes of the most suspicious Bitcoin transactions among the bank transactions. Let's look closely at every coincidence

In [35]:
columns = ['time','filer_org_name','beneficiary_bank','bank_usd','btc_usd','difference','Sender']
susp_trans = pd.DataFrame(columns=columns)

In [36]:
for number, (ind_bank, ind_btc) in enumerate(similar.items()):
    susp_trans.loc[number] = [btc_data.iloc[ind_btc]['time']] + data_map[['filer_org_name','beneficiary_bank','amount_transactions']].iloc[ind_bank].to_list() + \
            [btc_data.iloc[ind_btc]['btc_to_usd']] + [abs(btc_data.iloc[ind_btc]['btc_to_usd'] - data_map.iloc[ind_bank]['amount_transactions'])] + [btc_data.iloc[ind_btc]['Sender']]

The Bank of New York Mellon Corp. is the most frequent met bank related with suspicious botcoin transaction

In [37]:
susp_trans[susp_trans.filer_org_name == 'The Bank of New York Mellon Corp.']

,time,filer_org_name,beneficiary_bank,bank_usd,btc_usd,difference,Sender
0,2015-03-18,The Bank of New York Mellon Corp.,Credit Suisse AG,1762600.00,1.738694e+06,23906.136281,72020
2,2015-01-27,The Bank of New York Mellon Corp.,Credit Suisse AG,2000000.00,2.018702e+06,18702.338847,1149
3,2015-12-23,The Bank of New York Mellon Corp.,Kazkommertsbank,1899974.00,1.895907e+06,4066.933352,167065
7,2015-03-17,The Bank of New York Mellon Corp.,JSC Norvik Banka,1523991.16,1.525413e+06,1421.362459,138025
8,2016-02-16,The Bank of New York Mellon Corp.,Turkiye Cummuriyeti Ziraat Bankasi,1000000.00,1.007648e+06,7648.210381,82679
...,...,...,...,...,...,...,...
84,2015-07-29,The Bank of New York Mellon Corp.,Credit Suisse AG,1000000.00,1.013261e+06,13261.025245,34713
85,2015-08-04,The Bank of New York Mellon Corp.,Kbl European Private Bankers S.A.,1055229.52,1.076557e+06,21327.813098,86593
86,2015-07-03,The Bank of New York Mellon Corp.,Credit Suisse AG,1082851.49,1.091379e+06,8527.618859,702
87,2016-12-16,The Bank of New York Mellon Corp.,"Norvik Banka, JSC",1662500.00,1.683118e+06,20617.970942,54828


'Deutsche Bank AG'

In [38]:
susp_trans[susp_trans.filer_org_name == 'Deutsche Bank AG']

,time,filer_org_name,beneficiary_bank,bank_usd,btc_usd,difference,Sender
1,2016-12-30,Deutsche Bank AG,DBS Bank Ltd,4000000.0,4.013596e+06,13595.676693,107063
21,2016-02-01,Deutsche Bank AG,PT Bank Windu Kentjana Internation,1172683.0,1.167022e+06,5661.412355,105702


'The Northern Trust Company'

In [39]:
susp_trans[susp_trans.filer_org_name == 'The Northern Trust Company']

,time,filer_org_name,beneficiary_bank,bank_usd,btc_usd,difference,Sender
4,2015-01-26,The Northern Trust Company,Bank J Safra Sarasin Ltd,1000000.0,999823.008932,176.991068,82898


'China Investment Corporation'

In [40]:
susp_trans[susp_trans.filer_org_name == 'China Investment Corporation']

,time,filer_org_name,beneficiary_bank,bank_usd,btc_usd,difference,Sender
5,2016-12-01,China Investment Corporation,HSBC Bank Plc,140.44,1.404249e+02,0.015087,151840
19,2016-01-15,China Investment Corporation,Saigon Thuong Tin Commercial Joint Stock Bank,1000000.00,1.021339e+06,21338.946410,155855
20,2016-02-24,China Investment Corporation,Banque Pour Le Commerce Exterieur Lao Public,1388230.00,1.385127e+06,3102.795845,171081
26,2016-04-08,China Investment Corporation,Branch Banking And Trust Company,1779903.99,1.769298e+06,10605.803719,98686


'Barclays Plc'

In [41]:
susp_trans[susp_trans.filer_org_name == 'Barclays Plc']

,time,filer_org_name,beneficiary_bank,bank_usd,btc_usd,difference,Sender
6,2015-07-07,Barclays Plc,Barclays,1999887.04,1.979862e+06,20024.702388,115193


'Citigroup, Inc.'

In [42]:
susp_trans[susp_trans.filer_org_name == 'Citigroup, Inc.']

,time,filer_org_name,beneficiary_bank,bank_usd,btc_usd,difference,Sender
11,2015-12-01,"Citigroup, Inc.",Harris Na,120133.2,134873.114122,14739.914122,19485
12,2015-12-01,"Citigroup, Inc.",Norvik Banka JSC,274000.0,292966.439124,18966.439124,103630


'Standard Chartered Plc'

In [43]:
susp_trans[susp_trans.filer_org_name == 'Standard Chartered Plc']

,time,filer_org_name,beneficiary_bank,bank_usd,btc_usd,difference,Sender
15,2015-03-19,Standard Chartered Plc,Standard Chartered Bank Hong Kong Ltd,1251050.00,1.258085e+06,7034.964997,133584
16,2015-05-01,Standard Chartered Plc,Standard Chartered Bank Hong Kong Ltd,1374000.62,1.381535e+06,7534.280822,72020


'Fifth Third Bank, National Association'

In [45]:
susp_trans[susp_trans.filer_org_name == 'Fifth Third Bank, National Association']

,time,filer_org_name,beneficiary_bank,bank_usd,btc_usd,difference,Sender
25,2015-03-18,"Fifth Third Bank, National Association",Fifth Third Bank,1245000.0,1.246980e+06,1980.351292,11984


'HSBC Holdings Plc'

In [46]:
susp_trans[susp_trans.filer_org_name == 'HSBC Holdings Plc']

,time,filer_org_name,beneficiary_bank,bank_usd,btc_usd,difference,Sender
89,2016-03-24,HSBC Holdings Plc,HSBC Hong Kong,1654939.98,1.651695e+06,3244.754643,140560


In summary, using Nearest Neighbours algorithm and information about transactions it turned out to find 90 Bitcoin transactions that can be related with bank transactions

# Intersting observations

Let's consider the dataset with bitcoin transactions more closely. At first, let's find a Sender who does transactions the most frequently

In [47]:
btc_data.groupby('Sender').count().sort_values(by='time', ascending=False).head(5)

,time,Receiver,price,Trans_amount_BTC,btc_to_usd,unix
Sender,,,,,,
64103,10521,10521,10521,10521,10521,10521
33883,2151,2151,2151,2151,2151,2151
123040,1607,1607,1607,1607,1607,1607
154891,1210,1210,1210,1210,1210,1210
118823,1191,1191,1191,1191,1191,1191


As we can see the most frequent sender is a sender numer 64103 (this number is a decoded name). Let's consider his transactions

In [48]:
btc_data[btc_data.Sender == 64103].time.unique()

array(['2017-11-18T00:00:00.000000000'], dtype='datetime64[ns]')

In [49]:
btc_data[btc_data.Sender == 64103]['Trans_amount_BTC'].sum()

263.261872

In [50]:
btc_data[btc_data.Sender == 64103]['btc_to_usd'].sum()

2030610.3463954162

It looks suspiciously. This sender  transfered  263.261872 Bitcoin (2 030 610 USD) with 10521 different transactions at one day. Unfourtunetly, suggested bank transaction dataset has the last date 2017-09-05, so we can't find intersections with bank transactions.